# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [25]:
# Selenium - ferramenta que permite controlar o navegador (precisa do webdriver - o que permite o selenium conversar com o computador) )
# Webdriver ->Chrome -> chromedriver, Firefox ->gekodriver

# procurar no google e fazer download do webdriver 

# 4 comandos com o selenium
# criar um navegador
# entrar em sites
# encontrar informações /itens no site
#fechar o navegador


#Importar o selenium 
# !pip install selenium --user

In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

navegador=webdriver.Chrome()
# navegador=webdriver.Chrome("chromedriver.exe") # -> quando o chromedriver estiver no mesmo local do arquivo do programa


#Passo 1: Obter a cotação do dolar

navegador.get("https://www.google.pt/?hl=pt-PT")
navegador.find_element(By.XPATH, '//*[@id="L2AGLb"]/div').click()


navegador.find_element(By.XPATH,
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dolar real brasileiro")

navegador.find_element(By.XPATH,
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_dolar=navegador.find_element(By.XPATH,
                       '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_dolar)


# Passo 2: Obter a cotação do euro

navegador.get("https://www.google.pt/?hl=pt-PT")
#navegador.find_element(By.XPATH, '//*[@id="L2AGLb"]/div').click()


navegador.find_element(By.XPATH,
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro real brasileiro")

navegador.find_element(By.XPATH,
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro=navegador.find_element(By.XPATH,
                       '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_euro)


# Passo 3: Obter a cotação do ouro

navegador.get("https://www.melhorcambio.com/ouro-hoje")


cotacao_ouro=navegador.find_element(By.XPATH,
                       '//*[@id="comercial"]').get_attribute('value')

cotacao_ouro=cotacao_ouro.replace(",",".")

print(cotacao_ouro)

navegador.quit()


5.2360999999999995
5.98803014
308.73


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [27]:
# Passo 4: Importar a base e atualizar as cotações na minha base
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [35]:
# Passo 5 : Calcular os novos preços e salvar/ exportar a base de dados

#tabela.loc[linha,coluna]
#tabela.loc[0,'Cotação']=float(cotacao_dolar)
#tabela.loc[1,'Cotação']=float(cotacao_euro)

# posso em vez da linha posso colocar uma condição
 #tabela['Moeda']=='Dólar' - forma de obter as linhas que quero

tabela.loc[tabela['Moeda']=='Dólar','Cotação']=float(cotacao_dolar)
tabela.loc[tabela['Moeda']=='Euro','Cotação']=float(cotacao_euro)
tabela.loc[tabela['Moeda']=='Ouro','Cotação']=float(cotacao_ouro)

# Se a moeda estiver escrita de maneira diferente coloco os termos numa lista
#tabela['Moeda'] in ["Dólar", "dolar", "EUA"]

#display(tabela)

# atualizar as colunas
# preço de compra=Preço original * Cotação

tabela["Preço de Compra"]=tabela["Preço Original"]* tabela["Cotação"]

# preço de venda =Preço de compra * margem

tabela["Preço de Venda"]=tabela["Preço de Compra"]* tabela["Margem"]

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.23610,5236.047639,1.40,7330.466695
1,Carro Renault,4500.00,Euro,5.98803,26946.135630,2.00,53892.271260
2,Notebook Dell,899.99,Dólar,5.23610,4712.437639,1.70,8011.143986
3,IPhone,799.00,Dólar,5.23610,4183.643900,1.70,7112.194630
4,Carro Fiat,3000.00,Euro,5.98803,17964.090420,1.90,34131.771798
5,Celular Xiaomi,480.48,Dólar,5.23610,2515.841328,2.00,5031.682656
6,Joia 20g,20.00,Ouro,308.73000,6174.600000,1.15,7100.790000


### Agora vamos exportar a nova base de preços atualizada

In [37]:
tabela.to_excel("Produtos Novo.xlsx", index=False)